In [2]:
import re
import qiskit
import json
import time

import numpy as np
from typing import List
from qiskit import transpile
import time

from fast_generator import fc_tree_commute_recur_lookahead_fast

from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
from circuit_generator import construct_qcc_circuit

import numpy as np
# from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

from qiskit.transpiler import CouplingMap


In [3]:
#Compare json files in a specific folder
import os, json
def run_experiment_folder(folder_path = None, filename = None, save_output = False, threshold = 1):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)

            # Function to check if a string contains only 'I' characters
            def is_all_identity(pauli):
                return all(char == 'I' for char in pauli)

            # Filter the list to remove all identity Paulis
            test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for our method
            start_time = time.time()
            opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            opt_qc_f = transpile(opt_qc_f, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=CouplingMap().from_grid(11,11))
            end_time = time.time()
            our_time = end_time - start_time

            start_time = time.time()
            opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=CouplingMap().from_grid(11,11))
            end_time = time.time()
            combined_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": len(test_paulis),
                "times": {
                    "our_time": our_time,
                    "combined_time": combined_time
                },
                "gate_counts": {
                    "our_method": opt_qc_f.count_ops().get('cx', 0),
                    "combined_method": opt_qiskit.count_ops().get('cx', 0)
                },
                "circuit_entangling_depth": {
                    "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
                    "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1)
                },
                "test_paulis_file": f'benchmarks/results_hardware_topology_ibm_grid/test_new_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results_hardware_topology_ibm_grid/test_new_' + filename, 'w') as paulis_file:
                    json.dump([test_paulis, results], paulis_file, indent=4)
    

    

In [4]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, threshold = 1, filename = "Paulis"):

    results = []

    def is_all_identity(pauli):
        return all(char == 'I' for char in pauli)
    paulis = test_paulis
    # Filter the list to remove all identity Paulis
    test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]

    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    opt_qc_f = transpile(opt_qc_f, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=CouplingMap().from_grid(11,11))
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=CouplingMap().from_grid(11,11))
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0)
        },
        "circuit_entangling_depth": {
            "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
            "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1)
        },
        "test_paulis_file": f'benchmarks/results_hardware_topology_ibm_grid/test_new_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results_hardware_topology_ibm_grid/test_new_' + filename, 'w') as paulis_file:
            json.dump([test_paulis, results], paulis_file, indent=4)
    return sorted_entanglers_f
    

    

In [5]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,]# 12]
orbitals_list = [4, 6, 8, 12, 16, 20,]# 24]
#First evaluate the UCCSD ansatz:
for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = True, threshold = 1, filename=f"Paulis{len(test_paulis)}.json")

{'num_paulis': 24, 'times': {'our_time': 0.26618003845214844, 'combined_time': 0.18530488014221191}, 'gate_counts': {'our_method': 75, 'combined_method': 26}, 'circuit_entangling_depth': {'our_method': 55, 'combined_method': 21}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_new_Paulis24.json'}
{'num_paulis': 80, 'times': {'our_time': 1.125511884689331, 'combined_time': 0.6386773586273193}, 'gate_counts': {'our_method': 392, 'combined_method': 159}, 'circuit_entangling_depth': {'our_method': 284, 'combined_method': 115}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_new_Paulis80.json'}
{'num_paulis': 320, 'times': {'our_time': 4.987126350402832, 'combined_time': 3.1851730346679688}, 'gate_counts': {'our_method': 1953, 'combined_method': 780}, 'circuit_entangling_depth': {'our_method': 1255, 'combined_method': 559}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_new_Paulis320.json'}
{'num_paulis': 1656, 'times'

In [6]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = True, threshold = 1)

benzene.json
{'num_paulis': 1254, 'times': {'our_time': 24.15445637702942, 'combined_time': 19.85309910774231}, 'gate_counts': {'our_method': 11866, 'combined_method': 6302}, 'circuit_entangling_depth': {'our_method': 6189, 'combined_method': 4189}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_new_benzene.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 2.4414782524108887, 'combined_time': 1.8510229587554932}, 'gate_counts': {'our_method': 1327, 'combined_method': 531}, 'circuit_entangling_depth': {'our_method': 801, 'combined_method': 395}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_new_H2O.json'}
LiH.json
{'num_paulis': 61, 'times': {'our_time': 0.6832056045532227, 'combined_time': 0.466752290725708}, 'gate_counts': {'our_method': 280, 'combined_method': 140}, 'circuit_entangling_depth': {'our_method': 202, 'combined_method': 125}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_new_LiH.json'}


In [7]:
#Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, threshold = 1)

max_cut_benchmark_connected_n10_e12_l1.json
{'num_paulis': 22, 'times': {'our_time': 0.4657871723175049, 'combined_time': 0.29920005798339844}, 'gate_counts': {'our_method': 275, 'combined_method': 47}, 'circuit_entangling_depth': {'our_method': 135, 'combined_method': 37}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_new_max_cut_benchmark_connected_n10_e12_l1.json'}
max_cut_benchmark_connected_n10_e12_l3.json
{'num_paulis': 66, 'times': {'our_time': 0.8587360382080078, 'combined_time': 0.7409992218017578}, 'gate_counts': {'our_method': 635, 'combined_method': 130}, 'circuit_entangling_depth': {'our_method': 313, 'combined_method': 81}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_new_max_cut_benchmark_connected_n10_e12_l3.json'}
max_cut_benchmark_connected_n15_e63_l1.json
{'num_paulis': 78, 'times': {'our_time': 3.8816189765930176, 'combined_time': 3.702141046524048}, 'gate_counts': {'our_method': 674, 'combined_method': 197}, 'cir

In [8]:
#since in the maxcut problems there are many commuting terms and it's slow to search through all of them,
#we can set the threshold to 2 to early stop the search once found a simplified pauli of weight 2
#you can see the run time difference, for the second benchmark runtime reduces from 30second to 7 seconds.

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, threshold = 2)

max_cut_benchmark_connected_n10_e12_l1.json
{'num_paulis': 22, 'times': {'our_time': 0.29023218154907227, 'combined_time': 0.22538995742797852}, 'gate_counts': {'our_method': 254, 'combined_method': 47}, 'circuit_entangling_depth': {'our_method': 125, 'combined_method': 34}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_new_max_cut_benchmark_connected_n10_e12_l1.json'}
max_cut_benchmark_connected_n10_e12_l3.json
{'num_paulis': 66, 'times': {'our_time': 0.7121310234069824, 'combined_time': 0.5943779945373535}, 'gate_counts': {'our_method': 554, 'combined_method': 120}, 'circuit_entangling_depth': {'our_method': 261, 'combined_method': 85}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_new_max_cut_benchmark_connected_n10_e12_l3.json'}
max_cut_benchmark_connected_n15_e63_l1.json
{'num_paulis': 78, 'times': {'our_time': 2.470388412475586, 'combined_time': 2.1392982006073}, 'gate_counts': {'our_method': 735, 'combined_method': 227}, 'circu

In [9]:
#Labs

run_experiment_folder(folder_path = "benchmarks/labs_paulis", save_output = True, threshold= 2)

labs_n10_layers1.json
{'num_paulis': 80, 'times': {'our_time': 1.631622314453125, 'combined_time': 1.6366586685180664}, 'gate_counts': {'our_method': 814, 'combined_method': 261}, 'circuit_entangling_depth': {'our_method': 440, 'combined_method': 199}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_new_labs_n10_layers1.json'}
labs_n10_layers3.json
{'num_paulis': 240, 'times': {'our_time': 4.797212600708008, 'combined_time': 3.7140450477600098}, 'gate_counts': {'our_method': 2285, 'combined_method': 739}, 'circuit_entangling_depth': {'our_method': 1162, 'combined_method': 523}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_grid/test_new_labs_n10_layers3.json'}
labs_n11_layers1.json
{'num_paulis': 106, 'times': {'our_time': 2.9770455360412598, 'combined_time': 2.518239736557007}, 'gate_counts': {'our_method': 1133, 'combined_method': 351}, 'circuit_entangling_depth': {'our_method': 562, 'combined_method': 258}, 'test_paulis_file': 'benchmarks/resu